In [50]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web
from datetime import date as dt

In [51]:
# set parameters. Choose your security, benchmark, risk free rate (or proxy for risk free rate), start & end dates for the downloaded data
RISKY_ASSET = 'AXP'
START_DATE = '2013-03-31'
END_DATE = dt.today()

In [52]:
# five factors
factor_5_df = web.DataReader("F-F_Research_Data_5_Factors_2x3",
                             "famafrench",
                             start=START_DATE,
                             end=END_DATE)[0]
factor_5_df.iloc[-6:]

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2022-09,-9.35,-0.97,0.06,-1.51,-0.84,0.19
2022-10,7.83,1.86,8.05,3.07,6.52,0.23
2022-11,4.60,-2.67,1.38,6.01,3.11,0.29
2022-12,-6.41,-0.16,1.32,0.09,4.19,0.33
2023-01,6.65,4.43,-4.05,-2.62,-4.53,0.35
2023-02,-2.58,0.59,-0.80,0.92,-1.53,0.34


In [53]:
# create data frame of timeseries for asset, benchmark, and risk free rate proxy
df = yf.download([RISKY_ASSET],
                 start=START_DATE,
                 end=END_DATE,
                 progress=False,
                 auto_adjust=True)
df

,Open,High,Low,Close,Volume
Date,,,,,
2013-04-01,58.190936,58.475764,57.837063,58.052841,3012600
2013-04-02,58.156411,58.484397,57.914741,58.380821,3921600
2013-04-03,58.571257,58.597226,57.168868,57.350658,5102000
2013-04-04,57.497827,57.965290,57.393951,57.766190,3633900
2013-04-05,56.935134,56.961110,56.069464,56.528275,6898300
...,...,...,...,...,...
2023-04-10,158.039993,161.169998,157.800003,161.139999,2372600
2023-04-11,161.669998,162.580002,160.320007,161.830002,2742300
2023-04-12,163.220001,163.220001,158.869995,159.289993,2817600


In [54]:
# calculate returns
y = df['Close'].resample('M') \
    .last() \
    .pct_change() \
    .dropna()

y.index = y.index.to_period("m")
y.name = "ret"
y.iloc[-6:]

Date
2022-11    0.061569
2022-12   -0.062440
2023-01    0.188063
2023-02   -0.005374
2023-03   -0.051957
2023-04   -0.006822
Freq: M, Name: ret, dtype: float64

In [55]:
# join dataframes and rename your columns
factor_5_df = factor_5_df.join(y)
factor_5_df.columns = [
    "mkt", "smb", "hml", "rmw", "cma", "rf", "ret"
]

factor_5_df.iloc[-6:]

,mkt,smb,hml,rmw,cma,rf,ret
Date,,,,,,,
2022-09,-9.35,-0.97,0.06,-1.51,-0.84,0.19,-0.112434
2022-10,7.83,1.86,8.05,3.07,6.52,0.23,0.104564
2022-11,4.60,-2.67,1.38,6.01,3.11,0.29,0.061569
2022-12,-6.41,-0.16,1.32,0.09,4.19,0.33,-0.062440
2023-01,6.65,4.43,-4.05,-2.62,-4.53,0.35,0.188063
2023-02,-2.58,0.59,-0.80,0.92,-1.53,0.34,-0.005374


In [56]:
# normalize mkt factor
factor_5_df.loc[:, factor_5_df.columns != "ret"] /= 100

# calculate excess return 
factor_5_df["excess_ret"] = (
    factor_5_df["ret"] - factor_5_df["rf"]
)

In [57]:
# perform OLS regression and print summary
fama_french_model = smf.ols(formula = "excess_ret ~ mkt + smb + hml + rmw + cma",
                            data = factor_5_df).fit()
print(fama_french_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_ret   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     35.32
Date:                Sat, 15 Apr 2023   Prob (F-statistic):           1.51e-21
Time:                        01:28:06   Log-Likelihood:                 198.17
No. Observations:                 118   AIC:                            -384.3
Df Residuals:                     112   BIC:                            -367.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0027      0.004      0.611      0.5